In [1]:
""" 3/26/18 
With a functional mister_ed package, it's time to run the madry challenge.

Step 1 is to build a defended network that can transfer to their tensorflow nets.
Because Madry uses tensorflow, transferability is a must. 

From Madry:
The model is a residual convolutional neural network consisting of five residual units and a fully connected layer. 
This architecture is derived from the "w32-10 wide" variant of the Tensorflow model repository. 
The network was trained against an iterative adversary that is allowed to perturb each pixel by at most epsilon=8.0.
"""
None

In [2]:
# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn

import config
import prebuilt_loss_functions as plf
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain


In [3]:
# Step -1: load the classifier, data_loader, and the normalizer 
use_gpu = True
classifier_net = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
                                                           use_gpu=use_gpu)
classifier_net.eval()

val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          use_gpu=use_gpu)

cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                             std=config.CIFAR10_STDS)


Files already downloaded and verified


In [4]:
# Step 0: initialize hyperparams 
PGD_L_INF = 8.0 / 255.0
PGD_STEP_SIZE = 1.0 / 255.0
PGD_NUM_ITER = 16

In [5]:
# Step 1: setup loss object 
standard_xentropy = plf.VanillaXentropy(classifier_net, normalizer=cifar_normer)

In [6]:
# Step 2: build attack object 
pgd_attack_object = aa.LInfPGD(classifier_net, cifar_normer, standard_xentropy)

In [7]:
# Step 3: setup a batch of examples 
ex_minibatch, ex_targets = next(iter(val_loader))


In [8]:
pgd_attack_object._dtype

torch.FloatTensor

In [9]:
# Step 4: perform the actual attack 
pgd_adv_images = pgd_attack_object.attack(ex_minibatch, ex_targets, 
                                          l_inf_bound=PGD_L_INF,
                                          step_size=PGD_STEP_SIZE,
                                          num_iterations=PGD_NUM_ITER,
                                          random_init=True,
                                          signed=True,
                                          verbose=True)

(START):  87.5 correct

 81.25 correct
(iteration 00):  31.25 correct
(iteration 01):  0.0 correct
(iteration 02):  0.0 correct
(iteration 03):  0.0 correct
(iteration 04):  0.0 correct
(iteration 05):  0.0 correct
(iteration 06):  0.0 correct
(iteration 07):  0.0 correct
(iteration 08):  0.0 correct
(iteration 09):  0.0 correct
(iteration 10):  0.0 correct
(iteration 11):  0.0 correct
(iteration 12):  0.0 correct
(iteration 13):  0.0 correct
(iteration 14):  0.0 correct
(iteration 15):  0.0 correct
